In [1]:
import requests
import numpy as np
import pandas as pd

In [2]:
file_count = 12
row_count = int(42448764 / file_count)
file_count, row_count

(12, 3537397)

In [ ]:
for fn in range(1, file_count + 1):
    df_sample = pd.read_csv(
        './data/2019-Oct.csv', 
        skiprows=range(1, 1 + (row_count * (fn - 1))),
        nrows=row_count
    )
    df_sample.to_parquet(
        f'./data/2019-Oct-{fn}.parquet.gzip', 
        compression='gzip'
    )

In [42]:
df_sample_12 = pd.read_parquet(
    './data/2019-Oct-12.parquet.gzip'
)

In [43]:
df_sample_12.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3537397 entries, 0 to 3537396
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 242.9+ MB


In [44]:
df_sample_12.sample(1)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
669080,2019-10-29 15:06:13 UTC,view,12100811,2053013555816432043,kids.skates,xiaomi,377.13,565451513,8040d1a2-285c-475d-a13d-54477c00eba1


In [31]:
# df_sample_12['event_time'] = pd.to_datetime(df_sample_12['event_time'])

In [32]:
# view: 0, cart: 1, remove_from_cart: 2, purchase: 3
# df_sample_12.loc[df_sample_12['event_type'] == 'view', 'event_type'] = 0
# df_sample_12.loc[df_sample_12['event_type'] == 'cart', 'event_type'] = 1
# df_sample_12.loc[df_sample_12['event_type'] == 'remove_from_cart', 'event_type'] = 2
# df_sample_12.loc[df_sample_12['event_type'] == 'purchase', 'event_type'] = 3
# df_sample_12['event_type'] = pd.to_numeric(df_sample_12['event_type'], downcast='unsigned')

In [35]:
# df_sample_12['product_id'] = pd.to_numeric(df_sample_12['product_id'], downcast='unsigned')

In [36]:
# df_sample_12['category_id'] = pd.to_numeric(df_sample_12['category_id'], downcast='unsigned')

In [40]:
# df_sample_12['category_id'].nunique()

603

In [41]:
# df_sample_12['category_code'].nunique()

123

In [37]:
df_sample_12.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3537397 entries, 0 to 3537396
Data columns (total 9 columns):
 #   Column         Dtype              
---  ------         -----              
 0   event_time     datetime64[ns, UTC]
 1   event_type     uint8              
 2   product_id     uint32             
 3   category_id    uint64             
 4   category_code  object             
 5   brand          object             
 6   price          float64            
 7   user_id        int64              
 8   user_session   object             
dtypes: datetime64[ns, UTC](1), float64(1), int64(1), object(3), uint32(1), uint64(1), uint8(1)
memory usage: 205.8+ MB


In [38]:
df_sample_12.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-29 04:49:36+00:00,0,4700478,2053013560899928785,auto.accessories.videoregister,sho-me,141.91,514013862,53cfcfc2-671d-4e90-a21a-221d32cab9bc
1,2019-10-29 04:49:36+00:00,0,4000189,2053013566142809077,construction.tools.generator,tefal,167.29,529024659,24d5149d-7fca-4bb6-9a01-16c3072eaf15
2,2019-10-29 04:49:36+00:00,0,5701166,2053013553970938175,auto.accessories.player,None,144.12,563540902,cc9e09e5-775e-4485-9765-c106c8956c7b
3,2019-10-29 04:49:36+00:00,0,12718704,2053013553559896355,None,yokohama,71.95,552114951,95ef058c-3514-4c8d-9342-1c3e8ee206ab
4,2019-10-29 04:49:36+00:00,0,23301542,2053013561956893455,None,None,30.63,519648035,30fda761-bd5a-4c4f-a3dd-944e474bb054


## 데이터 분리

### 이벤트 데이터

In [47]:
df_event = df_sample_12[['event_time', 'event_type', 'product_id', 'user_session']].copy(deep=True)

In [56]:
df_event.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3537397 entries, 0 to 3537396
Data columns (total 4 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   event_time    object
 1   event_type    object
 2   product_id    int64 
 3   user_session  object
dtypes: int64(1), object(3)
memory usage: 108.0+ MB


### 유저 데이터

In [49]:
df_user = df_sample_12[['user_session', 'user_id']].drop_duplicates().copy(deep=True)

In [55]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 795172 entries, 0 to 3537390
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_session  795172 non-null  object
 1   user_id       795172 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 18.2+ MB


### 프로덕트 데이터

In [50]:
df_product = df_sample_12[['product_id', 'category_id', 'category_code', 'brand', 'price']].drop_duplicates().copy(deep=True)

In [57]:
df_product.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124492 entries, 0 to 3537317
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   product_id     124492 non-null  int64  
 1   category_id    124492 non-null  int64  
 2   category_code  60640 non-null   object 
 3   brand          95490 non-null   object 
 4   price          124492 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 5.7+ MB


## 전체 데이터 분리

In [69]:
df_event, df_user, df_product = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
cols_groups = [
    ['event_time', 'event_type', 'product_id', 'user_session'], 
    ['user_session', 'user_id'], 
    ['product_id', 'category_id', 'category_code', 'brand', 'price']
]

for fn in range(1, file_count + 1):
    df_sample = pd.read_parquet(f'./data/2019-Oct-{fn}.parquet.gzip')
    
    if df_event.empty:
        df_event = df_sample[cols_groups[0]].copy(deep=True)
    else:
        df_event = pd.concat([
            df_event,
            df_sample[cols_groups[0]].copy(deep=True)
        ])
        
    if df_user.empty:
        df_user = df_sample[cols_groups[1]].copy(deep=True)
    else:
        df_user = pd.concat([
            df_user,
            df_sample[cols_groups[1]].copy(deep=True)
        ])
    df_user = df_user.drop_duplicates()
        
    if df_product.empty:
        df_product = df_sample[cols_groups[2]].copy(deep=True)
    else:
        df_product = pd.concat([
            df_product,
            df_sample[cols_groups[2]].copy(deep=True)
        ])
    df_product = df_product.drop_duplicates()
    
    

In [70]:
df_event

,event_time,event_type,product_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
...,...,...,...,...
3537392,2019-10-31 23:59:58 UTC,view,2300275,22c57267-da98-4f28-9a9c-18bb5b385193
3537393,2019-10-31 23:59:58 UTC,view,10800172,5054190a-46cb-4211-a8f1-16fc1a060ed8
3537394,2019-10-31 23:59:58 UTC,view,5701038,05b6c62b-992f-4e8e-91f7-961bcb4719cd
3537395,2019-10-31 23:59:59 UTC,view,21407424,4c14bf2a-2820-4504-929d-046356a5a204


In [71]:
df_user

,user_session,user_id
0,72d76fde-8bb3-4e00-8c23-a032dfed738c,541312140
1,9333dfbd-b87a-4708-9857-6336556b0fcc,554748717
2,566511c2-e2e3-422b-b695-cf8e6e792ca8,519107250
3,7c90fc70-0e80-4590-96f3-13c02c18c713,550050854
4,c6bd7419-2748-4c56-95b4-8cec9ff8b80d,535871217
...,...,...
3537356,341a87d8-8cf4-4b4a-8b7d-749380697af1,566280860
3537365,b1df0fce-6e03-47cc-b777-782b77b1b100,532572658
3537384,2169c430-9acc-4db8-8061-ac70ff0c66ea,557202408
3537388,fb46b2fb-493b-477c-8d18-ea1c24c04020,544578298


In [72]:
df_product

,product_id,category_id,category_code,brand,price
0,44600062,2103807459595387724,None,shiseido,35.79
1,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20
2,17200506,2053013559792632471,furniture.living_room.sofa,None,543.10
3,1307067,2053013558920217191,computers.notebook,lenovo,251.74
4,1004237,2053013555631882655,electronics.smartphone,apple,1081.98
...,...,...,...,...,...
3536299,19000980,2053013557225718275,None,None,67.12
3536447,19001117,2053013557225718275,None,None,79.30
3536509,47000083,2110937189394154243,None,canon,1667.47
3536724,1701321,2053013553031414015,computers.peripherals.monitor,acer,123.04


## 전체 데이터 다운캐스트

### 이벤트 데이터 다운캐스트

In [73]:
df_event.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42448764 entries, 0 to 3537396
Data columns (total 4 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   event_time    object
 1   event_type    object
 2   product_id    int64 
 3   user_session  object
dtypes: int64(1), object(3)
memory usage: 1.6+ GB


In [76]:
df_event['event_time'] = pd.to_datetime(df_event['event_time'])

In [77]:
# view: 0, cart: 1, remove_from_cart: 2, purchase: 3
df_event.loc[df_event['event_type'] == 'view', 'event_type'] = 0
df_event.loc[df_event['event_type'] == 'cart', 'event_type'] = 1
df_event.loc[df_event['event_type'] == 'remove_from_cart', 'event_type'] = 2
df_event.loc[df_event['event_type'] == 'purchase', 'event_type'] = 3
df_event['event_type'] = pd.to_numeric(df_event['event_type'], downcast='unsigned')

In [78]:
df_event['product_id'] = pd.to_numeric(df_event['product_id'], downcast='unsigned')

In [79]:
df_event.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42448764 entries, 0 to 3537396
Data columns (total 4 columns):
 #   Column        Dtype              
---  ------        -----              
 0   event_time    datetime64[ns, UTC]
 1   event_type    uint8              
 2   product_id    uint32             
 3   user_session  object             
dtypes: datetime64[ns, UTC](1), object(1), uint32(1), uint8(1)
memory usage: 1.1+ GB


In [85]:
df_event.head()

,event_time,event_type,product_id,user_session
0,2019-10-01 00:00:00+00:00,0,44600062,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00+00:00,0,3900821,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01+00:00,0,17200506,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01+00:00,0,1307067,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04+00:00,0,1004237,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [91]:
df_event.to_parquet(
    f'./data/2019-Oct-event.parquet.gzip', 
    compression='gzip'
)

### 유저 데이터 다운캐스트

In [74]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9244772 entries, 0 to 3537390
Data columns (total 2 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   user_session  object
 1   user_id       int64 
dtypes: int64(1), object(1)
memory usage: 211.6+ MB


In [82]:
df_user['user_id'] = pd.to_numeric(df_user['user_id'], downcast='unsigned')
df_user = df_user.set_index('user_session')

In [83]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9244772 entries, 72d76fde-8bb3-4e00-8c23-a032dfed738c to 6764041a-9285-4869-8a32-a79adf31d212
Data columns (total 1 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  uint32
dtypes: uint32(1)
memory usage: 105.8+ MB


In [84]:
df_user.head()

,user_id
user_session,
72d76fde-8bb3-4e00-8c23-a032dfed738c,541312140
9333dfbd-b87a-4708-9857-6336556b0fcc,554748717
566511c2-e2e3-422b-b695-cf8e6e792ca8,519107250
7c90fc70-0e80-4590-96f3-13c02c18c713,550050854
c6bd7419-2748-4c56-95b4-8cec9ff8b80d,535871217


In [92]:
df_user.to_parquet(
    f'./data/2019-Oct-user.parquet.gzip', 
    compression='gzip'
)

### 프로덕트 데이터 다운캐스트

In [75]:
df_product.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 371536 entries, 0 to 3537027
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   product_id     371536 non-null  int64  
 1   category_id    371536 non-null  int64  
 2   category_code  187788 non-null  object 
 3   brand          290716 non-null  object 
 4   price          371536 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 17.0+ MB


In [95]:
df_product['product_id'] = pd.to_numeric(df_product['product_id'], downcast='unsigned')

In [96]:
df_product['category_id'] = pd.to_numeric(df_product['category_id'], downcast='unsigned')

In [108]:
df_product['price'] = pd.to_numeric(df_product['price'], downcast='float')

In [99]:
df_product = df_product.set_index('product_id')

In [109]:
df_product.info()

<class 'pandas.core.frame.DataFrame'>
UInt64Index: 371536 entries, 44600062 to 13100770
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   category_id    371536 non-null  uint64 
 1   category_code  187788 non-null  object 
 2   brand          290716 non-null  object 
 3   price          371536 non-null  float32
dtypes: float32(1), object(2), uint64(1)
memory usage: 12.8+ MB


In [110]:
df_product.head()

,category_id,category_code,brand,price
product_id,,,,
44600062,2103807459595387724,None,shiseido,35.790001
3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.200001
17200506,2053013559792632471,furniture.living_room.sofa,None,543.099976
1307067,2053013558920217191,computers.notebook,lenovo,251.740005
1004237,2053013555631882655,electronics.smartphone,apple,1081.979980


In [111]:
df_product.to_parquet(
    f'./data/2019-Oct-product.parquet.gzip', 
    compression='gzip'
)

In [86]:
df_product['category_id'].nunique(), df_product['category_code'].nunique(), df_product['brand'].nunique()

(624, 126, 3444)

In [117]:
df = df_product[['category_id', 'category_code']].drop_duplicates().sort_values('category_id').copy(deep=True)

In [118]:
df

,category_id,category_code
product_id,,
8600174,2053013552226107603,appliances.environment.fan
8500097,2053013552259662037,None
6200260,2053013552293216471,appliances.environment.air_heater
3900821,2053013552326770905,appliances.environment.water_heater
4200541,2053013552351936731,appliances.environment.air_conditioner
...,...,...
20400129,2172371118332051820,apparel.shoes
20700006,2172371122509578610,apparel.shoes
1201496,2172371436436455782,electronics.tablet


In [121]:
df = df.set_index('category_id')

In [122]:
df

,category_code
category_id,
2053013552226107603,appliances.environment.fan
2053013552259662037,None
2053013552293216471,appliances.environment.air_heater
2053013552326770905,appliances.environment.water_heater
2053013552351936731,appliances.environment.air_conditioner
...,...
2172371118332051820,apparel.shoes
2172371122509578610,apparel.shoes
2172371436436455782,electronics.tablet


In [134]:
df[df['category_code'].apply(lambda x: x is None)]

,category_code
category_id,
2053013552259662037,None
2053013552385491165,None
2053013552427434207,None
2053013552469377249,None
2053013552662315243,None
...,...
2162513024617808683,None
2162513070000177858,None
2162513074060264222,None
